In [1]:
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
from dblib import mssql
import mmodb
print('done')

Trying to connect to database s30.mmo ....
done


In [2]:
#load data
#mv_graph_plat_cam_net.sql
sql = "select " \
    " ifca, intertidalfid, season, speciesid, n_sum, n_count as n_count" \
    " from v_aggregate_all_tally_all" \
    " where isnull(n_count,0)>0" \
    " order by ifca, intertidalfid, season, speciesid, n_sum desc"
print(sql)
with mssql.Conn('mmo', 's30', user='sa', pw='GGM290471') as cnn:
    df = pd.read_sql(sql, cnn)
df['n_sum_log'] = np.log(df['n_count'])
list(df)

select  ifca, intertidalfid, season, speciesid, n_sum, n_count as n_count from v_aggregate_all_tally_all where isnull(n_count,0)>0 order by ifca, intertidalfid, season, speciesid, n_sum desc


['ifca',
 'intertidalfid',
 'season',
 'speciesid',
 'n_sum',
 'n_count',
 'n_sum_log']

In [17]:
#BY SPECIES/SEASON

In [18]:
df['rank'] = df.groupby(['ifca'])['n_sum_log'].transform(lambda x: pd.cut(x, 3, labels=range(1,4), duplicates='drop'))
df.to_csv('C:/Users/Graham Monkman/OneDrive/Documents/MMOMapping/jupyter/outputs/mmo_species_rank.csv')
df.to_sql('df_mmo_species_rank', mmodb.ENGINE, if_exists='replace')

print('Done')

Done


In [ ]:
#NWW OVERALL

In [12]:
sql = "select" \
    " ifca, intertidalfid, season_full, count(*) as post_n" \
    " from v_all" \
    " group by ifca, intertidalfid, season_full" \
    " having count(*) > 0" \
    " order by ifca, intertidalfid"

print(sql)
with mssql.Conn('mmo', 's30', user='sa', pw='GGM290471') as cnn:
    df_all = pd.read_sql(sql, cnn)
    
df_all['n_count_log'] = np.log( df_all['post_n'])
list(df_all)

select ifca, intertidalfid, season_full, count(*) as post_n from v_all group by ifca, intertidalfid, season_full having count(*) > 0 order by ifca, intertidalfid


['ifca', 'intertidalfid', 'season_full', 'post_n', 'n_count_log']

In [ ]:
#OVERALL

In [15]:
#ranks within season
#df_all['rank_total'] = df_all.groupby(['ifca', 'season_full'])['post_n'].transform(lambda x: pd.qcut(x, 3, labels=range(1,4), duplicates='drop'))

#ranks across season
df_all['rank_total'] = df_all.groupby(['ifca'])['post_n'].transform(lambda x: pd.qcut(x, 3, labels=range(1,4), duplicates='drop'))

In [16]:
df_all.to_csv('C:/Users/Graham Monkman/OneDrive/Documents/MMOMapping/jupyter/outputs/mmo_overall_rank_by_season_full.csv')
df_all.to_sql('df_mmo_season_activity_rank', mmodb.ENGINE, if_exists='replace')

In [20]:
np.log(1000)

6.907755278982137